<h1> Circuit Node Classification using Graph Convolutional Networks </h1>

In [1]:
#importing dependencies

import numpy as np
import os
import networkx as nx
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

from spektral.layers import GCNConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

from collections import Counter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# parse the data

In [4]:
import sys
from spektral.data import Dataset, Graph
sys.path.append('../lib')
from data_pre_processing import load_data
sys.path.remove('../lib')
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess

class CircuitDataset(Dataset):
    def read(self):
        circuits = []
        for circ in ['c6288', 'c17', 'c5315']:
            A, X, labels = load_data(circ, '../data/output', normalize="")
            circuits.append(Graph(x=X.toarray(), a=A, y=labels))
        return circuits

dataset = CircuitDataset(normalize_x=True, transforms=[LayerPreprocess(GCNConv)])
dataset

(6288, 6288)
(6288, 4)
(17, 17)
(17, 4)
(5315, 5315)
(5315, 4)


CircuitDataset(n_graphs=3)

In [18]:
dataset.a

### Create batches in disjoint mode

In [5]:
from spektral.data import DisjointLoader
loader = DisjointLoader(dataset, batch_size=3)

In [15]:
inputs, labels = loader.__next__()
X, A, i = inputs
N = X.shape[0]
F = X.shape[1]
N, F, A.shape, i.shape
num_classes = 2

In [16]:
num_classes

2

# GCN Model

In [17]:
# Parameters
channels = 16           # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 200            # Number of training epochs
es_patience = 10        # Patience for early stopping

# Preprocessing operations
# A = GCNConv.preprocess(A).astype('f4')

# Model definition
X_in = Input(shape=(F, ))
fltr_in = Input((N, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GCNConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])

dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GCNConv(num_classes,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
    log_dir='./Tensorboard_GCN_cora',
)
callback_GCN = [tbCallBack_GCN]

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 4)            0           input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 11620)]      0                                            
__________________________________________________________________________________________________
gcn_conv_1 (GCNConv)            (None, 16)           64          dropout_2[0][0]                  
                                                                 input_4[0][0]                